# Hello Mr. Malaz

I added `kaggle.json` to my `C:\Users\laila\.kaggle` so the code runs correctly. If you face any issues running the notebook, this might be it.

laila haddad 202010537

## 1 , 2 : Read and Process the Data

Installing the Kaggle API and Downloading the Dataset

In [2]:
%pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c titanic

import zipfile
with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


Loading the Dataset into a DataFrame

In [8]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
test_y = pd.read_csv('gender_submission.csv')

test_df = test_df.merge(test_y , on='PassengerId')

X_train = train_df.drop(columns=['Survived'])
y_train = train_df['Survived']

X_test = test_df.drop(columns=['Survived'])
y_test = test_df['Survived']

X_test = X_test[X_train.columns]


test_df.head()
train_df.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


remove unecessary columns

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess_data(train_df, test_df):

    # drop unecessary columns
    columns_to_drop = ['Name', 'Ticket', 'Cabin']
    train_df.drop(columns=columns_to_drop, inplace=True)
    test_df.drop(columns=columns_to_drop, inplace=True)
    
    # Handle missing values
    train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
    train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
    train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)
    
    test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
    test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)
    test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)
    
    # Feature scaling
    features_to_scale = ['Age', 'Fare']
    scaler = StandardScaler()
    train_df[features_to_scale] = scaler.fit_transform(train_df[features_to_scale])
    test_df[features_to_scale] = scaler.transform(test_df[features_to_scale])

    # One-hot encoding
    categorical_features = ['Sex', 'Embarked', 'Pclass']
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    train_encoded = encoder.fit_transform(train_df[categorical_features])
    test_encoded = encoder.transform(test_df[categorical_features])

    train_encoded_df = pd.DataFrame(train_encoded, index=train_df.index, columns=encoder.get_feature_names_out(categorical_features))
    test_encoded_df = pd.DataFrame(test_encoded, index=test_df.index, columns=encoder.get_feature_names_out(categorical_features))

    # Drop original categorical columns
    train_df.drop(columns=categorical_features, inplace=True)
    test_df.drop(columns=categorical_features, inplace=True)

    # Concatenate the encoded features with the original DataFrames
    train_df = pd.concat([train_df, train_encoded_df], axis=1)
    test_df = pd.concat([test_df, test_encoded_df], axis=1)

    # Drop duplicates
    train_df.drop_duplicates(inplace=True)
    test_df.drop_duplicates(inplace=True)
    
    return train_df, test_df


train_df, test_df = preprocess_data(train_df, test_df)

X_train = train_df.drop(columns=['Survived'])
y_train = train_df['Survived']

X_test = test_df.drop(columns=['Survived'])
y_test = test_df['Survived']

X_test = X_test[X_train.columns]

print(X_train.head())
print(X_test.head())


   PassengerId       Age  SibSp  Parch      Fare  Sex_female  Sex_male  \
0            1 -0.565736      1      0 -0.502445         0.0       1.0   
1            2  0.663861      1      0  0.786845         1.0       0.0   
2            3 -0.258337      0      0 -0.488854         1.0       0.0   
3            4  0.433312      1      0  0.420730         1.0       0.0   
4            5  0.433312      0      0 -0.486337         0.0       1.0   

   Embarked_C  Embarked_Q  Embarked_S  Pclass_1  Pclass_2  Pclass_3  
0         0.0         0.0         1.0       0.0       0.0       1.0  
1         1.0         0.0         0.0       1.0       0.0       0.0  
2         0.0         0.0         1.0       0.0       0.0       1.0  
3         0.0         0.0         1.0       1.0       0.0       0.0  
4         0.0         0.0         1.0       0.0       0.0       1.0  
   PassengerId       Age  SibSp  Parch      Fare  Sex_female  Sex_male  \
0          892  0.394887      0      0 -0.490783         0.0 

C:\Users\laila\AppData\Local\Temp\ipykernel_24256\4269825088.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
C:\Users\laila\AppData\Local\Temp\ipykernel_24256\4269825088.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

## 3 - Model Experimentation :  

Importing Necessary Libraries

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, precision_recall_curve


### SGDClassifier

In [47]:
sgd_clf = SGDClassifier(random_state=42)

sgd_params = {
    'loss': ['hinge', 'log_loss', 'modified_huber'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000]
}

sgd_grid = GridSearchCV(sgd_clf, sgd_params, cv=5, scoring='accuracy')
sgd_grid.fit(X_train, y_train)

print(f"Best hyperparameters for SGDClassifier: {sgd_grid.best_params_}")

y_pred_sgd = sgd_grid.predict(X_test)

precision_sgd = precision_score(y_test, y_pred_sgd)
recall_sgd = recall_score(y_test, y_pred_sgd)

print(f"SGDClassifier Precision: {precision_sgd}")
print(f"SGDClassifier Recall: {recall_sgd}")

Best hyperparameters for SGDClassifier: {'alpha': 0.0001, 'loss': 'log_loss', 'max_iter': 1000}
SGDClassifier Precision: 0.0
SGDClassifier Recall: 0.0


c:\Users\laila\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forest Classifier


In [48]:

rf_clf = RandomForestClassifier(random_state=42)

rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf_grid = GridSearchCV(rf_clf, rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)

print(f"Best hyperparameters for RandomForestClassifier: {rf_grid.best_params_}")


Best hyperparameters for RandomForestClassifier: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}


In [50]:

y_pred_rf = rf_grid.predict(X_test)

precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)

print(f" Precision: {precision_rf}")
print(f"Recall: {recall_rf}")


 Precision: 0.7971014492753623
Recall: 0.7236842105263158


lofistic regression

In [11]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42, max_iter=1000)

log_reg_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='accuracy')
log_reg_grid.fit(X_train, y_train)

print(f"Best hyperparameters for LogisticRegression: {log_reg_grid.best_params_}")

y_pred_log_reg = log_reg_grid.predict(X_test)

precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)

print(f"LogisticRegression Precision: {precision_log_reg}")
print(f"LogisticRegression Recall: {recall_log_reg}")


Best hyperparameters for LogisticRegression: {'C': 0.1, 'solver': 'liblinear'}
LogisticRegression Precision: 0.9235668789808917
LogisticRegression Recall: 0.9539473684210527


## 4 - Model Evaluation : 

## Conclusion

### SGDClassifier
**Precision**: 1.0
**Recall**: 0.013

### RandomForestClassifier
**Precision**:  0.79
**Recall**:  0.72

### LogisticRegressionClassifier
**Precision**:  0.92
**Recall**:  0.9
The LogisticRegressionClassifier model did much better, giving the highest precision and recall scores. it is the best fit because it handles binary classification tasks very well which is exactly what we are trying ti achieve here
